In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=service, options=options)

# Ganti dengan URL halaman review toko di Tokopedia
url = "https://www.tokopedia.com/dkmall/review"

driver.get(url)

reviews = []

while True:
    # Ambil elemen review
    time.sleep(3)  # Tunggu halaman masuk
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
        review_elements = soup.findAll("article", attrs={'class':'css-ccpe8t'})

        for review in review_elements:
            username = review.find("span", attrs={"class":"name"}).text
            comment = review.find("span", attrs={"data-testid":"lblItemUlasan"}).text
            product_container = review.findAll("a",attrs={"class":"styProduct"})

            for element in product_container:
                product = review.find("p", attrs={"data-unify":"Typography"}).text
                
                reviews.append({
                    "username": username,
                    "review": comment,
                    "product": product
                })
    except:
        print("there is error")
        
    # Cek apakah ada tombol "Next" untuk lanjut ke halaman berikutnya
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Laman berikutnya"]')
        next_button.click()
        time.sleep(2)
    except:
        print("No more pages.")
        break  # Keluar jika tidak ada tombol "Next"

driver.quit()

# Simpan ke CSV
df = pd.DataFrame(reviews)
df.to_csv("tokopedia_review.csv", index=False)
print("Scraping selesai! Data disimpan dalam tokopedia_reviews.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_3716\339290948.py:27: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  review_elements = soup.findAll("article", attrs={'class':'css-ccpe8t'})
C:\Users\user\AppData\Local\Temp\ipykernel_3716\339290948.py:32: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  product_container = review.findAll("a",attrs={"class":"styProduct"})


there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
there is error
No more pages.
Scraping selesai! Data disimpan dalam tokopedia_reviews.csv


In [24]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

# Load data hasil scraping
df = pd.read_csv("tokopedia_review.csv")

# Tampilkan contoh data
print("data dari csv: ")
print(df.head())

# Inisialisasi stopword remover & stemmer
stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

# Fungsi preprocessing
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Hapus karakter spesial
    text = stopword_remover.remove(text)  # Hapus stopword
    text = stemmer.stem(text)  # Stemming
    return text

# Terapkan ke data
df["clean_review"] = df["review"].astype(str).apply(clean_text)

slang_dict = {
    "yg": "yang",
    "bgt": "banget",
    "tdk": "tidak",
    "gk": "gak",
    "trs": "terus",
    "bnr2": "benar-benar",
    "kayu2": "kayu-kayu",
    "sukaaa": "sukaaa",
    "jd" : "jadi",
    "byk2" : "banyak",
    "mantaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaap": "mantap",
    "bsnahabbaabananananananananananananananannaannaananan":""
}

def normalize_slang(text):
    words = text.split()
    words = [slang_dict[word] if word in slang_dict else word for word in words]
    return " ".join(words)

df["clean_review"] = df["clean_review"].apply(normalize_slang)
print(df.head())

data dari csv: 
  username                                             review  \
0    ANDRI  mendarat dengan aman, packing aman, engga boco...   
1    K***s  thank you, hotto nya asli, pengiriman cepat, h...   
2   Awalia                                             mantap   
3    gugun                                               good   
4  Marissa                                     sesuai pesanan   

                                             product  
0  Autumn 7% Glycolic Acid Toner Solution Exfolia...  
1  Hotto Purto Multigrain Superfood Purple Potato...  
2  Azarine Calm My Acne Sunscreen Moisturiser SPF...  
3  Ms Glow Men Energizer Facial Wash / Face Wash ...  
4   Golden Viera Soap Original Asli Paket Hemat 3pcs  
  username                                             review  \
0    ANDRI  mendarat dengan aman, packing aman, engga boco...   
1    K***s  thank you, hotto nya asli, pengiriman cepat, h...   
2   Awalia                                             mantap   
3  

In [25]:
import spacy

nlp = spacy.load("xx_ent_wiki_sm")  # Model NER multibahasa

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

df["entities"] = df["clean_review"].astype(str).apply(extract_entities)
print(df.head())

  username                                             review  \
0    ANDRI  mendarat dengan aman, packing aman, engga boco...   
1    K***s  thank you, hotto nya asli, pengiriman cepat, h...   
2   Awalia                                             mantap   
3    gugun                                               good   
4  Marissa                                     sesuai pesanan   

                                             product  \
0  Autumn 7% Glycolic Acid Toner Solution Exfolia...   
1  Hotto Purto Multigrain Superfood Purple Potato...   
2  Azarine Calm My Acne Sunscreen Moisturiser SPF...   
3  Ms Glow Men Energizer Facial Wash / Face Wash ...   
4   Golden Viera Soap Original Asli Paket Hemat 3pcs   

                                        clean_review  \
0    darat aman packing aman engga bocor kirim cepat   
1  thank you hotto nya asli kirim cepat harga sah...   
2                                             mantap   
3                                               

In [27]:
import networkx as nx
from pyvis.network import Network

G = nx.Graph()

for index, row in df.iterrows():
    user = row["username"]
    review = row["clean_review"]
    entities = row["entities"]

    # Tambahkan node user
    G.add_node(user, type="User")

    # Tambahkan node entitas (Produk/Sentimen)
    for ent_text, ent_label in entities:
        G.add_node(ent_text, type=ent_label)
        G.add_edge(user, ent_text, relation="review")  # Hubungkan user dengan entitas

# Visualisasi Knowledge Graph
net = Network(notebook=True, height="800px", width="100%", bgcolor="#222222", font_color="white")
net.from_nx(G)

# Tambahkan fitur interaktif
net.toggle_physics(True)  # Aktifkan fisika agar node tidak tumpang tindih
net.force_atlas_2based(gravity=-50, central_gravity=0.01)  # Sesuaikan gaya tarik antar node

# Simpan ke file HTML dan tampilkan
net.show("knowledge_graph.html")

knowledge_graph.html


In [28]:
# Convert nodes and edges into sentences
def graph_to_text(graph):
    text_data = []
    for node in graph.nodes():
        relations = [f"{node} is related to {nbr} via {graph[node][nbr]['relation']}"
                     for nbr in graph.neighbors(node)]
        text_data.extend(relations)
    return text_data

text_corpus = graph_to_text(G)
print(text_corpus)

['ANDRI is related to aman via review', 'ANDRI is related to engga bocor kirim cepat via review', 'aman is related to ANDRI via review', 'aman is related to Vica via review', 'aman is related to Raflie via review', 'aman is related to S***l via review', 'engga bocor kirim cepat is related to ANDRI via review', 'K***s is related to thank you via review', 'K***s is related to asli kirim via review', 'thank you is related to K***s via review', 'asli kirim is related to K***s via review', 'Marissa is related to sesuai pesan via review', 'sesuai pesan is related to Marissa via review', 'sesuai pesan is related to Zulham via review', 'C***y is related to benar-benar via review', 'C***y is related to gak via review', 'benar-benar is related to C***y via review', 'gak is related to C***y via review', 'd***f is related to wangi via review', 'wangi is related to d***f via review', 'wangi is related to Sulthan via review', 'wangi is related to S***a via review', 'wangi is related to R***n via rev

In [29]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text corpus to embeddings
embeddings = embedder.encode(text_corpus)
embeddings = np.array(embeddings).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance (cosine similarity)
index.add(embeddings)

print(f"FAISS Index contains {index.ntotal} vectors")

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

FAISS Index contains 632 vectors


In [35]:
def retrieve_knowledge(query, top_k=3):
    query_embedding = embedder.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    
    retrieved_info = [text_corpus[idx] for idx in indices[0] if idx < len(text_corpus)]
    return retrieved_info

query = "Review bagus"
retrieved = retrieve_knowledge(query)
print("Retrieved Knowledge:", retrieved)

Retrieved Knowledge: ['bagus is related to S***i via review', 'S***i is related to bagus via review', 'bagus is related to Bayu via review']
